Tasks:
Embedding Similarity

1. Write a function that takes as input two sentences, finds identical words, computes their contextual embeddings and prints the cosine similarity.
Ask interesting questions (case, punctuation,...)
Use longer words and find similarity between sub words

2. Load the IMBD reviews and sample 500 positive and negative reviews.
Extract the embedding vectors
Fit a Naive Bayes classifier


In [2]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 
     |████████████████████████████████| 84 kB 2.4 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np
from scipy.spatial.distance import cosine
from transformers import AutoTokenizer
#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
from transformers import pipeline
from transformers import TFAutoModel
#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "bert-base-uncased"
model = TFAutoModel.from_pretrained(checkpoint)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
##Task 1
def similarity_check(sent1, sent2):
    split1 = sent1.split()
    split2 = sent2.split()
    same_words = []
    for i in split1: 
        if split2.count(i) > 0: same_words.append([split1.index(i), split2.index(i), i])
    
    raw_inputs = [sent1, sent2]
    inputs = tokenizer(raw_inputs, return_tensors="tf")
    outputs = model(inputs)

    similarity = []
    for index, i in enumerate(same_words):
        embedd_1 = outputs.last_hidden_state[0,same_words[index][0], :]
        embedd_2 = outputs.last_hidden_state[1,same_words[index][1], :]
        similarity.append([same_words[index][2], 1 - cosine(embedd_1, embedd_2)])
    return similarity

In [14]:
sent1 = "work to live"
sent2 = "live to work"
similarity_check(sent1, sent2)

[['work', 0.13984434306621552],
 ['to', 0.5869054198265076],
 ['live', 0.3014388978481293]]

In [3]:
##Task 2
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)
full_docs = np.concatenate((train_data, test_data))




In [4]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
dec_reviews = ["" for x in range(full_docs.shape[0])]
for j in range(full_docs.shape[0]):
    dec_reviews[j] = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in full_docs[j]])